# Pricing

One of the most important things we would want to do with a complex contract is price it.

How much should we sell the PPN we just constructed for? 


This is a massive simplification of section 4 of  

> [1] Jones, S. Peyton, Jean-Marc Eber, and Julian Seward. “Composing contracts: an adventure in financial engineering.” ACM SIG-PLAN Notices 35.9 (2000): 280-292.

```haskell
eval :: Model -> Currency -> Contract -> ValProc
```

In [6]:
from decimal import Decimal 
from datetime import date 
from contracts.processes import BinomialPriceProcess, ConstProcess 
from contracts import *

process =  BinomialPriceProcess(Decimal("100"),0.2)

In [7]:
from contracts.valuation import Evaluate


In [8]:
from contracts.plot import plot, plots
 

In [9]:
import plotly.graph_objects as go

In [10]:
btc_price = BinomialPriceProcess(Decimal("30000"),sigma=.2)
eth_price = BinomialPriceProcess(Decimal("2100"), sigma=.2)
cad_price = ConstProcess(Decimal("1"))
prices = {Asset.BTC:btc_price, Asset.ETH:eth_price, Asset.CAD:cad_price}

In [16]:
def prices():
 
    return {
        Asset.BTC:BinomialPriceProcess(Decimal("30000"),sigma=.2), 
        Asset.ETH:BinomialPriceProcess(Decimal("2100"), sigma=.2), 
        Asset.CAD:ConstProcess(Decimal("1")),
        Asset.USD:BinomialPriceProcess(Decimal("1.36"), sigma = 0.04)

    }

In [18]:
value = Evaluate(prices())

In [24]:
c1 = ONE(BTC)
c2 = GIVE(ONE(BTC)) 
plots({
    'long btc':value(c1),
    'short btc':value(c2), 
})

In [25]:
c1 = ONE(BTC) 
c2 = SCALE(CONST(20000), ONE(USD))
plots({
    'long btc':value(c1), 
    '$20,000 usd': value(c2)
})

In [26]:
c1 = ONE(BTC)
c2 = SCALE(CONST(Decimal("0.5")),ONE(BTC))
plots({'1 BTC':value(c1),'0.5 BTC':value(c2)})

In [28]:
def spottrade(left_amount:Decimal, left:Asset, right_amount:Decimal, right:Asset)->Contract:
    return AND(
        SCALE(CONST(left_amount),ONE(left)),
        GIVE(SCALE(CONST(right_amount),ONE(right)))
    )

def call(t:date,amount:Decimal, base:Asset, k:Decimal, quote:Asset)->Contract:
 
    return WHEN(
        t,
        OR(
            ZERO(),
            spottrade(amount,base, amount*k, quote)
        )
    )


In [29]:
btc = ONE(BTC)

option = call(date(2023,7,1),Decimal("10"), BTC, Decimal("30000"),CAD)


In [30]:
value = Evaluate(prices())
plots({
    'holding a btc':value(btc),'a call on 10 BTC':value(option)
    })

Important: We haven't yet computed the value of these contracts, we've just calculated the *intrinsic* value of the contract under one possible price path.

Options also have time value: to calculate this we might want to run this simulation multiple times and take a mean.

Another important caveat is that we haven't yet applied interest rate calculations. Money now is worth more than money in the future; our evaluation class should take in an interest rate process as well as FX processes, and apply it to the `WHEN` branch.